<img src="./logo_UNSAM.jpg" align="right" width="150" /> 

# TS5: Estimación espectral: Ancho de banda de señales reales 

**Alumnas:** María Victoria Poric y Agustina Rocío Paolini Rosso  
**Carrera:** Ingeniería Biomédica    
**Materia:** Análisis y Procesamiento de Señales    
**Fecha:** Octubre 2025 

## Consigna

Tomar del repositorio `PDStestbench` los siguientes tres tipos de señales registradas:

- Electrocardiograma (ECG): `ECG_TP4.mat` — Registro electrocardiográfico durante una prueba de esfuerzo.  
- Pletismografía (PPG): `PPG.csv` — Señal registrada en reposo.  
- Audio: Tres registros con voz y dos registros con silbidos.

1) Estimar la densidad espectral de potencia (PSD) de cada señal utilizando alguno de los métodos vistos en clase:
- Periodograma ventaneado  
- Welch  
- Blackman–Tukey

2) A partir de la PSD:
- Estimar el ancho de banda de cada señal.  
- Presentar los resultados en una tabla comparativa.

*Bonus*:
- Proponer y analizar una señal adicional, explicando su origen y cómo fue digitalizada. Repetir el análisis de PSD y ancho de banda.

## Introducción
El análisis espectral es una herramienta esencial en el procesamiento de señales, ya que permite estudiar cómo se distribuye la energía de una señal en el dominio de la frecuencia. En muchos casos, como en señales biomédicas o acústicas, la observación directa en el dominio temporal no es suficiente para comprender los fenómenos que intervienen en la generación de la señal. Por ejemplo, en un electrocardiograma (ECG) o una señal de pletismografía (PPG), los componentes de baja frecuencia están asociados a procesos fisiológicos lentos, mientras que las componentes de mayor frecuencia pueden corresponder a ruido o artefactos. En señales de audio, por otra parte, el contenido espectral define la calidad y las características tonales del sonido.

Para caracterizar cuantitativamente este contenido frecuencial se utiliza la Densidad Espectral de Potencia (Power Spectral Density, PSD), la cual describe cómo se distribuye la potencia de la señal en función de la frecuencia. En otras palabras, la PSD permite identificar qué frecuencias contienen mayor energía, ofreciendo una visión más profunda de la dinámica interna de la señal.

Matemáticamente, para una señal estacionaria $x(t)$ con transformada de Fourier $X(f)$, la PSD se define como:

$$
S_{xx}(f) = \lim_{T \to \infty} \frac{1}{T} \, E\left\{ \left| X_T(f) \right|^2 \right\} \tag{1}
$$

donde $X_T(f)$ es la transformada de Fourier de la señal truncada a un intervalo de duración $T$, y $E{⋅}$ denota el valor esperado. Sin embargo, en la práctica trabajamos con señales discretas y de duración finita, por lo que es necesario estimar la PSD a partir de los datos disponibles.

El **método de Welch** es una de las técnicas más utilizadas para estimar la PSD de manera robusta. Surge como una mejora del periodograma clásico, que aunque conceptualmente simple, presenta alta variabilidad en sus estimaciones. Welch propone reducir dicha varianza mediante un proceso de promediado:

1) *Segmentación*: la señal $x[n]$ se divide en varios segmentos solapados de longitud $L$.
   
2) *Ventaneo*: cada segmento se multiplica por una ventana (por ejemplo, Hamming o Hann) para minimizar las fugas espectrales causadas por la truncación.

3) *Cálculo del periodograma*: se obtiene la potencia espectral de cada segmento:
$$
P_k(f) = \frac{1}{L U} \left| \sum_{n=0}^{L-1} w[n] \, x_k[n] \, e^{-j 2 \pi f n} \right|^2 \tag{2}
$$
donde $w[n]$ es la ventana aplicada, $U$ un factor de normalización, y $x_k[n]$ es el segmento k-ésimo de la señal.  

4) *Promediado*: los periodogramas de todos los segmentos se promedian:
$$
S_{xx}(f) = \frac{1}{K} \sum_{k=1}^{K} P_k(f) \tag{3}
$$
obteniendo así una estimación más suave y confiable de la densidad espectral.

Este procedimiento mejora la estabilidad del resultado a costa de una ligera pérdida en la resolución en frecuencia, generando un equilibrio adecuado entre precisión y robustez.

El método de Welch resulta particularmente apropiado para señales fisiológicas y de audio, ya que estas suelen ser cuasi-estacionarias (su estadística cambia lentamente en el tiempo), y el promediado permite reducir los efectos del ruido y las variaciones transitorias.

A partir de la PSD estimada, es posible calcular el **ancho de banda efectivo** de cada señal, es decir, el rango de frecuencias donde se concentra la mayor parte de su energía. Este parámetro proporciona información relevante sobre la naturaleza de la señal y sobre los requisitos de muestreo y filtrado necesarios para su procesamiento.

Una forma común de estimarlo es identificar el intervalo de frecuencias $[f1,f2]$ que contiene un porcentaje significativo (por ejemplo, el 95%) de la potencia total:

$$
\int_{f_1}^{f_2} S_{xx}(f) \, df = 0.95 \int_{0}^{f_{\text{max}}} S_{xx}(f) \, df \tag{4}
$$

Comparar el ancho de banda entre señales de distinta naturaleza (ECG, PPG y audio) permite comprender sus diferencias fundamentales: mientras que las señales fisiológicas suelen concentrar su energía en frecuencias bajas, las señales acústicas presentan una distribución más amplia y rica en armónicos.

El objetivo principal de este trabajo es estimar y analizar la densidad espectral de potencia (PSD) de distintos tipos de señales, como la del electrocardiograma (ECG), pletismografía (PPG) y audio, utilizando el método de Welch como técnica de estimación. A partir de las PSD obtenidas, se busca determinar el ancho de banda característico de cada señal y comparar sus comportamientos espectrales, resaltando las diferencias entre señales fisiológicas y acústicas. De esta manera, el estudio permite no solo aplicar herramientas de procesamiento digital de señales vistas en clase, sino también interpretar los resultados desde un punto de vista práctico y fisiológico, comprendiendo cómo el contenido en frecuencia refleja las propiedades y el origen físico de cada señal.

---
## Análisis y desarrollo
En primer lugar, se cargaron tres tipos de señales:

- ECG (Electrocardiograma) con una frecuencia de muestreo de `fs = 1000 Hz`,

- PPG (Pletismografía) con una frecuencia de muestreo de `fs = 400 Hz`,

- Audio proveniente de un archivo .wav, del cual también se obtuvo automáticamente su frecuencia de muestreo.